In [1]:
SAVE_IMAGES = False
TEXT_ARXIU = 'corpus_raw.txt'

import re, spacy, pickle

from utils import EntitatNomenada, obtenir_totes_les_coincidencies, convertir_fragments_a_referencies, afegir_entitat_nomenada, comprova_si_solapa, entitats_nomenades_cmp
from spacy import displacy
from pathlib import Path

In [2]:
# Inicialitzem spacy en català.
nlp = spacy.load("ca_core_news_sm")

In [3]:
# Text on buscarem entitats nomenades.

with open(TEXT_ARXIU, 'rb') as file:
    text_sencer = file.read().decode('utf-8')

In [4]:
# Tokenitza el text.
doc_spacy = nlp(text_sencer)

# Obté els tokens com strings.
tokens = [token.text for token in doc_spacy]

In [5]:
# Inicialitzem una llista buida on anirem guardant les entitats
# nomenades resultants.
entitats_nomenades = []

In [6]:
# Definim l'expressió regular per trobar dates senceres i hores
regex_dates_hores = (r'\b(?:0?[1-9]|[12][0-9]|3[01])[-/](?:0?[1-9]|1[0-2])([-/](\d{2}|\d{4}))?\b|' # 23/02/2024
    r'\b(?:0?[1-9]|[12][0-9]|3[01])\s(de\s|d\')?(?:gener|febrer|març|abril|maig|juny|juliol|agost|setembre|octubre|novembre|desembre)(\s(del?\s)?(\d{2,4}))?\b|' # 25 de novembre de 2015
    r'\b(([Ee]l\s)?gener|([Ee]l\s)?febrer|([Ee]l\s)?març|([Ll]\')?abril|([Ee]l\s)?maig|([Ee]l\s)?juny|([Ee]l\s)?juliol|([Ll]\')?agost|([Ee]l\s)?setembre|([Ll]\')?octubre|([Ee]l\s)?novembre|([Ee]l\s)?desembre)(\s(del?\s)(\d{2,4}))\b|' # El març del 2019
    r'\b([01]?\d|2[0-3])([:\.][0-5]\d\s?)(hores|h)?\b|' # 23.15 h
    r'\b(([Ee]l\s)|[Ll]\'any\s)?\d{4}\b|' # El 1995
    r'\b[Ss]egle\sM{0,3}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})\b|' # Segle XXI
    r'\d{1,2}\-[GFMAJSOND]') # 12-J

# Per obtenir les strings completes de cada data trobada
dates_senceres_hores = obtenir_totes_les_coincidencies(regex_dates_hores, text_sencer)

# Imprimim les dates trobades
print("Dates i hores trobades:", "\n".join(dates_senceres_hores), sep="\n\n")

Dates i hores trobades:

l'any 1986
1999
1989
1991
1993
1994
1996
1999
1998
L'octubre del 1998
26 de desembre
24 de maig
1995
13-M
31 de maig de 1938
1999
4 d'abril
29 de març
4 d'abril
el 1936
el 1936
1859
24 de juliol
l'any 1998
l'any 2000
l'abril del 2001
1845
1902
segle XIX
29 de juny
7 de juny
11 de setembre
1845
1902
l'any 1855
El 1870
1977
l'any 1874
1877
1893
1886
El 1945
2000
1999
2000
1998
el 2000
1999
el 1999
17 d'abril
1999
5 de setembre
11 de setembre
25 de juny de 1852
2/4
11-S
1999
8 d'agost
l'any 1995
l'any 2004
20 de setembre de 1999
1999
3 de desembre de 1999
15.21 hores
21.21
23.20 hores
22.00 hores
febrer del 1999
el 2000
1999
l'any 2000
el 1999
el 2000
el 1999
El 2000
23 de febrer
28 de febrer
28 de febrer
28 de febrer
1 de març
el 1999
el 1998
7 d'octubre
1 de novembre
El març del 1999
el 1998
1870
1917
1914
1925
juliol de 1995
el 2005
21 de juny
21 de juny
18 de juny
20 de juny
el maig del 1998
4 de març de 1998
el 1988
el 1991
març del 1998
19 de març de 1998
19

In [7]:
# Tokenitza les troballes.
# Converteix aixó en una llista de llistes de tokens en format string.
troballes_tokenitzades = [[t.text for t in nlp(trob)] for trob in dates_senceres_hores]

# Obtenir les referencies com a instancies d'EntitatNomenada.
entitats_nomenades += convertir_fragments_a_referencies(tokens, troballes_tokenitzades, 'DATE')

# Ara verifiquem si aquestes es corresponen correctament als tokens inicials.
for en in entitats_nomenades:
    en.verificar(tokens)
else:
    print('Tot verificat!')

Tot verificat!


In [8]:
# Definim l'expressió regular per trobar moments i dates relatius al dia actual
regex_dates_relatives = (r'\b((?:[Aa]quest\s)?([Dd]illuns|[Dd]imarts|[Dd]imecres|[Dd]ijous|[Dd]ivendres|[Dd]issabte|[Dd]iumenge)(?:\sque\sve|\spassat)?)\b|' # Aquest dilluns / Dimecres / diumenge que ve
    r'\b([Aa]questa\ssetmana|[Aa]quest\smes|[Aa]quest\sany)\b|' # Aquest mes
    r'\b(([Ll]a\ssetmana\s|[Ee]l\smes\s|[Ll]\'any\s)(que\sve|passa(t|da)|precedent|següent|anterior|posterior))\b|' # L'any que ve
    r'\b([Ff]a\s|[Dd]\'aquí\sa?\s?|[Cc]ada\s?)?([Gg]airebé\s|[Tt]ot\sjust\s|([Pp]oc\s)?més\s(d\'|de\s))?([Uu]na?e?s?\s|[Dd](o|ue)s\s|[Tt]res\s|[Qq]uatre\s|[Cc]inc\s|[Ss]is\s)(di(a|es)|setman(a|es)|meso?s?|anys?)\b|' # Fa gairebé dos anys
    r'\b([Aa]vui|[Dd]emà|([Aa]bans-d\')?[Aa]hir|[Dd]emà\spassat)\b|' # Abans-d'ahir
    r'\b(?:[Jj]ust\s)?[Aa]ra|(?:[Mm]olt\s|[Pp]oc\s|[Jj]ust\s)?[Aa]bans|(?:(?:[Pp]oc\s|[Jj]ust\s)?[Dd]esprés)\b|' # Poc abans
    r'\b[AaEe]\s?l\'?\s(mes\s(de\s|d\'))?(gener|febrer|març|abril|maig|juny|juliol|agost|setembre|octubre|novembre|desembre)\b|' # El mes de juny
    r'\b(((?:[Pp]rimera\s|[Úú]ltima\s)hora\s(?:del\s|de\sla\s))?|([Aa]quest\s|[AaEe]l\s|[Ll]a\s|[Aa]\sla\s)?)(matí|migdia|tarda|vespre|nit|matinada)|' # Aquesta matinada
    r'\b([Ll]a|[Ll]es)\s(una|dues|tres|quatre|cinc|sis|set|vuit|nou|deu|onze|dotze|d{1,2})(?:\s(del\smatí|del\smigdia|de\sla\starda|del\svespre|de\sla\snit|de\sla\smatinada))\b') # Les cinc de la tarda 

# Per obtenir les strings completes de cada data trobada
dates_relatives = obtenir_totes_les_coincidencies(regex_dates_relatives, text_sencer)

# Imprimim les EN trobades
print("Dates relatives trobades:", "\n".join(dates_relatives), sep="\n\n")

Dates relatives trobades:

després
ahir
un dia
després
ara
ara
abans
Ara
Ara
el gener
Ara
dijous
a la nit
ahir
dimecres
ahir
aquest any
ahir
ahir
ahir
ahir
abans
després
l'any passat
després
ahir
l'any anterior
sis anys
cada tres mesos
fa uns mesos
dos dies
després
fa uns mesos
tres anys
aquest any
dilluns
dijous
d'aquí a una setmana
abans
dilluns
Ahir
ahir
A la tarda
ahir
ara
ahir
demà
dimarts
dimecres
la setmana que ve
dissabte
avui
l'any següent
després
ahir
ahir
aquest divendres
ara
al novembre
Després
dos anys
ahir
fa uns dies
just abans
dilluns
a la nit
després
després
demà
l'any precedent
ahir
l'any passat
ara
Aquest any
aquest mes
ahir
aquest matí
aquest vespre
cinc anys
després
el maig
el juny
el setembre
dijous
A l juny
al juliol
després
el maig
el juny
ahir
després
ahir
ara
Avui
nit
la nit
avui
dilluns
ahir
les tres de la matinada
després
la nit
després
abans
al novembre
la setmana que ve
aquest mes
abans
al novembre
el maig
ahir
després
després
l'any passat
el gener
al gene

In [9]:
# Tokenitza les troballes.
# Converteix aixó en una llista de llistes de tokens en format string.
troballes_tokenitzades = [[t.text for t in nlp(trob)] for trob in dates_relatives]

# Obtenir les referencies com a instancies d'EntitatNomenada.
entitats_nomenades += convertir_fragments_a_referencies(tokens, troballes_tokenitzades, 'TIME')

# Ara verifiquem si aquestes es corresponen correctament als tokens inicials.
for en in entitats_nomenades:
    en.verificar(tokens)
else:
    print('Tot verificat!')

Tot verificat!


In [10]:
# Definim l'expressió regular per trobar freqüències
regex_frequencies = (r'\b(([Cc]ada|([Uu]na?|[Dd]os|[Dd]ues|[Tt]res|[Qq]uatre|[Cc]inc|[Ss]is|[Ss]et|[Vv]uit|[Nn]ou|[Dd]eu)\s(cop|vegada)\s(al\s|a\sla\s|(a\s)?l\'|per\s))'
    r'(dia|setmana|mes|any|estiu|tardor|hivern|primavera|dilluns|dimarts|dimecres|dijous|divendres|dissabte|diumenge)\b|' # Un cop a l'any
    r'\b[Ee]ls\s(dilluns|dimarts|dimecres|dijous|divendres|dissabte|diumenge)\b|' # Els dilluns
    r'\b([Dd]i[aà]ria?|[Ss]etmanal|[Qq]uinzenal|[Pp]eriòdica?|[Ff]reqüent|[Oo]casional|([Bb]i|[Tt]ri|[Qq]uadri|[Ss]e)?([Mm]ensuals?|mestrals?|[Aa]nuals?|ennals?))(ment)?)\b|' #Setmanalment / trimestral / biennal
    r'\b[Ss]empre|[Mm]ai|[Ss]ovint|(([Aa]|[Dd]e)\svegades)|[Dd]e\stant\sen\stant\b') # Sovint

# Per obtenir les strings completes de cada data trobada
frequencies = obtenir_totes_les_coincidencies(regex_frequencies, text_sencer)

# Imprimim les EN trobades
print("Expressions de freqüència trobades:", "\n".join(frequencies), sep="\n\n")

Expressions de freqüència trobades:

diari
sovint
sempre
sempre
mai
una vegada al dia
a vegades
diari
mai
Mai
Mai
Sempre
mai
mai
periòdicament
Diari
anual
mai
mai
mai
diari
de vegades
mai
ocasionalment
mai
sempre
anual
mai
sempre
mai
mai


In [11]:
# Tokenitza les troballes.
# Converteix aixó en una llista de llistes de tokens en format string.
troballes_tokenitzades = [[t.text for t in nlp(trob)] for trob in frequencies]

# Obtenir les referencies com a instancies d'EntitatNomenada.
entitats_nomenades += convertir_fragments_a_referencies(tokens, troballes_tokenitzades, 'TIME')

# Ara verifiquem si aquestes es corresponen correctament als tokens inicials.
for en in entitats_nomenades:
    en.verificar(tokens)
else:
    print('Tot verificat!')

DESCARTAT: ['Mai']
DESCARTAT: ['Mai']
DESCARTAT: ['periòdicament']
DESCARTAT: ['diari']
DESCARTAT: ['sempre']
Tot verificat!


In [12]:
# Definim l'expressió regular per detectar nombres ordinals
regex_ordinals = (r'\b([Pp]rimer(a|e?s)?|[Ss]egon(a|es)?|[Tt]ercer(a|e?s)?|[Qq]uart(a|e?s)?|[Cc]inqu[èe](ns|na|nes)?|[Ss]is[èe](ns|na|nes)?|([Dd]is)?[Ss]et[èe](ns|na|nes)?|'
    r'([Dd]i)?[Vv]uit[èe](ns|na|nes)?|([Dd]i)?[Nn]ov[èe](ns|na|nes)?|[Dd]es[èe](ns|na|nes)?|[Oo]nz[èe](ns|na|nes)?|[Dd]otz[èe](ns|na|nes)?|[Tt]retz[èe](ns|na|nes)?|'
    r'[Cc]atorz[èe](ns|na|nes)?|[Qq]uinz[èe](ns|na|nes)?|[Ss]etz[èe](ns|na|nes)|[Vv]int[èe](ns|na|nes)??)\b|' # Cinquè
    r'\b\d+(rs?|ts?|è|a|ns?|es)\.?\b|' # 4t
    r'\b(?=[MDCLXVI])M{0,3}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})\b(?=\s[A-Z])') # XXIII Jornades

# Per obtenir les strings completes de cada data trobada
ordinals = obtenir_totes_les_coincidencies(regex_ordinals, text_sencer)

# Imprimim les EN trobades
print("Ordinals trobats:", "\n".join(ordinals), sep="\n\n")

Ordinals trobats:

primer
Tercer
primer
segona
Primer
Primer
primer
tercer
Primera
segones
vintena
primer
primera
primers
primera
primera
primera
primera
quarta
primer
Primer
segon
onze
tercera
cinquè
primer
primera
segon
primer
desè
primer
primera
quart
quart
quart
primer
XXXVI
segon
primera
Cinquena
Cinquena
primera


In [13]:
# Tokenitza les troballes.
# Converteix aixó en una llista de llistes de tokens en format string.
troballes_tokenitzades = [[t.text for t in nlp(trob)] for trob in ordinals]

# Obtenir les referencies com a instancies d'EntitatNomenada.
entitats_nomenades += convertir_fragments_a_referencies(tokens, troballes_tokenitzades, 'ORDINAL')

# Ara verifiquem si aquestes es corresponen correctament als tokens inicials.
for en in entitats_nomenades:
    en.verificar(tokens)
else:
    print('Tot verificat!')

Tot verificat!


In [14]:
# Definim l'expressió regular per detectar monedes
regex_monedes = (r'(\d{1,3}(?:,\d{1,3})?\s?(\.\d{3})*(?:,\d{1,3})?\s?)(mil(ers)?\s(de\s)?)?([bm]ilions\s(de\s)?)?((cèntims?|centaus?|¢|penics?)|(afganis?|AFN)|(ariarys?|MGA)|'
    r'(฿|bahts?|THB)|(balboas?|PAB)|(birrs?|ETB)|(bolívars?|VEF)|(bolivianos?|BOB)|(₵|cedis?|GHS)|(₡|colons?|CRC)|(córdob(a|es)|NIO)|(coron(a|es)|DKK|SKK|EEK|ISK|NOK|SEK|CZK)|'
    r'(dalasis?|GMD)|(denars?|MKD)|(dinars?|DZD|BHD|IQD|JOD|KWD|LYD|TND|RSD|IRR)|(dírhams?|AED|MAD)|(dobr(a|es)|STD)|'
    r'(\$|\$CAN|dòlars?|AUD|BSD|BBD|BZD|BMD|BND|KYD|CAD|XCD|USD|FJD|GYD|HKD|JMD|LRD|NAD|NZD|SBD|SGD|SRD|TWD|TTD|ZWL)|(₫|dongs?|VND)|(₯|DR|dracm(a|es))|(drams?|AMD)|(escuts?|CVE)|'
    r'(M?€|d?\'?euros?|M?EUR)|(ƒ|FL|flor(í|ins)|ANF|AWG)|(fòrints?|HUF)|(₣|FB|FLUX|FS|francs?|BIF|XAF|XOF|XPF|KMF|CDF|DJF|GNF|RWF|CHF)|(gourdes?|HTG)|(guaranís?|PYG)|(₴|hrívni(a|es)|UAH)|'
    r'(¥|iens?|iuans?|renminbis?|JPY|CNY)|(kin(a|es)|PGK)|(₭|kips?|LAK)|(kun(a|es)|HRK)|(kwach(a|es)|MWK|ZMK)|(kwanz(a|es)|AOA)|(kyats?|MMK)|(laris?|GEL)|(lats|LVL)|(leks?|ALL)|'
    r'(lempir(a|es)|HNL)|(leones?|SLL)|(leus?|MDL|RON)|(levs?|BGN)|(lilangenis?|SZL)|([£₤]|lir(a|es)|TRY)|(litas|LTL)|([£₤]|£EG|£IR|£LIB|£SYR|£TQ|'
    r'(lliur(a|es)(\sesterlin(a|es))?|EGP|GBP|GIP|GGP|JEP|LBP|FKP|IMP|SHP|SYP|SDG|SSP))|(lotis?|LSL)|(manats?|AZN|TMT)|(DM|MF|marcs?|BAM)|(meticals?|MZN)|(₦|nair(a|es)|NGN)|'
    r'(nafk(a|es)|ERN)|(ngultrums?|BTN)|(ouguiy(a|es)|MRO)|(pa\'ang(a|ues)|TOP)|(pata(ca|ques)|MOP)|(₱|$MEX|$CH|CU|pesos?|ARS|COP|CUC|CUP|DOP|PHP|MXN|UYU|CLP)|'
    r'(pesset(a|es)|pt(a|es)\.?|ESP|M?PTA)|(pul(a|es)|BWP)|(quetzals?|GTQ)|(rands?|ZAR)|(R$|reals?|BRL)|(rials?|YER|IRR|OMR)|(៛|riels?|KHR)|(ringgits?|MYR)|(riyals?|QAR|SAR)|'
    r'(rubles?|BYR|RUB)|(₨|rupi(a|es)|INR|IDR|MVR|MUR|NPR|PKR|SCR|LKR)|(sols?(?=\s)|PEN)|(soms?|KGS|UZS)|(somonis?|TJS)|(sucres?)|(tak(a|es)|BDT)|(tal(a|es)|WST)|(tengu?es?|KZT)|'
    r'(₮|tögrögs?|MNT)|(tolars?)|(vatus?|VUV)|(₩|wons?|KPW|KRW)|(₪|£IS|(nous?\s)?xéquels?|ILS)|(xílings?|KES|SOS|TZS|UGX)|(zaires?)|(ZL|z[łl]otys?|PLN)|¤)')

# Per obtenir les strings completes de cada data trobada
monedes = obtenir_totes_les_coincidencies(regex_monedes, text_sencer)

# Imprimim les EN trobades
print("Monedes trobades:", "\n".join(monedes), sep="\n\n")

Monedes trobades:

180 milions de pessetes
42 milions d'euros
7.000 milions de pessetes
135.400 pessetes
467.800 milions de pessetes
2.000 milions de pessetes
56 milions d'euros
90.000 milions de dòlars
300 milions de pessetes
1,8 milions d'euros
8,01 milions d'euros
1.332 milions de pessetes
992 milions d'euros
164.981 milions de pessetes
591 milions d'euros
98.334 milions de pessetes
1.600 milions d'euros
266.218 milions de pessetes
4.500 milions de pessetes
27.081 milions de pessetes
91.440 milions de pessetes
15.095 milions de pessetes
26.033 milions de pessetes
2.209 milions de pessetes
352.000 milions de pessetes
20.831 milions de pessetes
89 milions de pessetes
1.500.000 euros
1.200 euros
16.000 milions de pessetes
42.000 euros
3.500 pessetes
3.500 pessetes
88.522 milions de pessetes
83.668 milions de pessetes
2,73 euros
2,99 euros
20 cèntims


In [15]:
# Tokenitza les troballes.
# Converteix això en una llista de llistes de tokens en format string.
troballes_tokenitzades = [[t.text for t in nlp(trob)] for trob in monedes]

# Obtenir les referències com a instàncies d'EntitatNomenada.
entitats_nomenades += convertir_fragments_a_referencies(tokens, troballes_tokenitzades, 'MONEY')

# Ara verifiquem si aquestes es corresponen correctament als tokens inicials.
for en in entitats_nomenades:
    en.verificar(tokens)
else:
    print('Tot verificat!')

Tot verificat!


In [16]:
# Expressió regular per detectar percentatges
regex_percentatges = (r'\-?(?:\d{1,3}(?:\.\d{3})*|\d+)(?:,\d+)?\s?(%|per\scent)|' # 23%
    r'\b(?:zero|u|dos|tres|quatre|cinc|sis|set|vuit|nou|deu|onze|dotze|tretze|catorze|quinze|setze|disset|divuit|dinou|vint|trenta|quaranta|cinquanta|seixanta|setanta|vuitanta|noranta|cent)\sper\scent\b') # tres per cent

# Per obtenir les strings completes de cada data trobada
percentatges = obtenir_totes_les_coincidencies(regex_percentatges, text_sencer)

# Imprimim les EN trobades
print("Mesures trobades:", "\n".join(percentatges), sep="\n\n")

Mesures trobades:

45,7%
100%
34%
0,5%
26%
23%
76%
73%
5,01%
7,53%
39,28%
0,4%
1,5%
5%
9%
18,7%
20,9%
2,6%
4,5%
-3%
34,7%
50%
9,1%
16%
74,5%
9,4%
56%
9,3%
1,6%
34%
2,8%
16,6%
7%
125%
13%
85 per cent
85 per cent
15 per cent
12,5%
25,2%
18,7%
3,5%
3,64%
85%
45,3%
9,5%
7%
7,6%
1,9%
6,1%
5%
3,6%
3,5%
5%
3%
9%
66%
11%


In [17]:
# Tokenitza les troballes.
# Converteix això en una llista de llistes de tokens en format string.
troballes_tokenitzades = [[t.text for t in nlp(trob)] for trob in percentatges]

# Obtenir les referències com a instàncies d'EntitatNomenada.
entitats_nomenades += convertir_fragments_a_referencies(tokens, troballes_tokenitzades, 'PERCENT')

# Ara verifiquem si aquestes es corresponen correctament als tokens inicials.
for en in entitats_nomenades:
    en.verificar(tokens)
else:
    print('Tot verificat!')

Tot verificat!


In [18]:
# Expressió regular per detectar unitats de mesura
regex_mesures = (r'((-?\d{1,3}(?:[.,]\d{3})*(?:,\d+)?\s*))\s?((([QRYZEPTGMKkhdcmµnpfzyrq]|da)?'
    r'((m|g|s|A|K|mol|cd|N|Pa|J|W|C|V|Ω|S|F|T|Wb|H|rad|sr|Hz|lm|lx|Bq|Gy|Sv|kat|B|b|L|l|cal|fg|mi|UA|ly|pc|t|HP|hp|ac|Ac|[°º]C|[°º]F|P|bar|atm)¯?[¹²³123]?([p\·\/]?([QRYZEPTGMKkhdcmµnpfazyrq]?(m|g|s|A|K|mol|cd|h)¯?[¹²³123]?))*))|'
    r'([Qq]uett[aà]|[Rr]onn[aà]|[Yy]ott[aà]|[Zz]ett[aà]|[Ee]x[aà]|[Pp]et[aà]|[Tt]er[aà]|[Gg]ig[aà]|[Mm]eg[aà]|[KkQq]u?il[oò]|[Hh]ect[oò]?|[Dd]ec[aà]|[Dd]ec[ií]|[Cc]ent[ií]|[Mm]il·l[ií]|[Mm]icr[oò]|[Nn]an[oò]|[Pp]ic[oò]|'
    r'[Ff]emt[oò]|[Aa]tt[oò]|[Zz]ept[oò]|[Yy]oct[oò]|[Rr]ont[oò]|[Qq]ect[oò])?(metres?|grams?|segons?|minuts?|hor(a|es)|amperes?|(graus?\s)kelvin|candel(a|es)|mols?|newtons?|pascals?|joules?|watts?|coulombs?|volts?|ohms?|siemens?|farads?|'
    r'tesl(a|es)|webers?|henrys?|radiant?s?|estereoradiant?s?|hertzs?|lumens?|luxs?|becquerels?|grays?|sieverts?|katals?|bels?|bytes?|bits?|litres?|gal[óo](ns)?|calori(a|es)|frigori(a|es)|mill(a|es)(\snàuti(ca|ques))?|'
    r'llegu(a|es)|iard(a|es)|peus?|polzad(a|es)|bra(ça|ces)|unitats?\sastronòmi(ca|ques)|anys?\sllum|pàrsecs?|ton(a|es)|tonelad(a|es)|un(ça|ces)|cavalls?|àre(a|es)|acres?|graus?|graus?\s(Celsius|centígrads?|Fahrenheits?)|poises?|'
    r'bars?|atmosfer(a|es)|psis?)(\sper)?\s?(metres?|grams?|segons?|amperes?|graus?\skelvin|candel(a|es)|mols?|hor(a|es))?((al)?\squadrats?|al\scub|cúbics?)?)\b')

# Per obtenir les strings completes de cada data trobada
mesures = obtenir_totes_les_coincidencies(regex_mesures, text_sencer)

# Imprimim les EN trobades
print("Mesures trobades:", "\n".join(mesures), sep="\n\n")

Mesures trobades:

500 grams 
21 hores 
200 metres per segon
-3 s
20 hores
00 hores
10 minuts
12 hores
000 l
14 hectàrees 
600 metres 
200 hectàrees
3.000 megawatts 


In [19]:
# Tokenitza les troballes.
# Converteix això en una llista de llistes de tokens en format string.
troballes_tokenitzades = [[t.text for t in nlp(trob)] for trob in mesures]

# Obtenir les referències com a instàncies d'EntitatNomenada.
entitats_nomenades += convertir_fragments_a_referencies(tokens, troballes_tokenitzades, 'QUANTITY')

# Ara verifiquem si aquestes es corresponen correctament als tokens inicials.
for en in entitats_nomenades:
    en.verificar(tokens)
else:
    print('Tot verificat!')

DESCARTAT: ['21', 'hores']
DESCARTAT: ['-', '3', 's']
DESCARTAT: ['20', 'hores']
DESCARTAT: ['00', 'hores']
DESCARTAT: ['000', 'l']
Tot verificat!


In [20]:
# Expressió regular per detectar quantitats
regex_quantitats = r'(\d+(?:[.,]\d{3})*(?:,\d+)?\s*)((?!(?:a|amb|en|entre|per|sense|sota|sobre|del?|pel|al|i|o|que)\s)[^\s,\.\)\%\;]+)(?=\s|,|\.|$)'

# Per obtenir les strings completes de cada data trobada
quantitats = obtenir_totes_les_coincidencies(regex_quantitats, text_sencer)

# Imprimim les EN trobades
print("Quantitats trobades:", "\n".join(quantitats), sep="\n\n")

Quantitats trobades:

1986
18 temes
23 anys
19 anys
19 anys
742 havia
500 grams
70
63 vots
1.701 vots
638
7.302 arquitectes
339
45
1998
1.052 arquitectes
1998
67 adolescents
320 persones
26
35 morts
24
1.300.000 persones
000
000
400.000 persones
100.000 assistents
15
000
20.000 persones
3.000 persones
2.000 persones
1995
400 ciutats
50
000
40
000
400
13-M
224 veïns
165 ferits
31
1938
80 edificis
180 milions
25 millors
1999
4 d'abril
29
4 d'abril
1936
1936:
14 parlamentaris
300 convidats
1859
100 dies
8 anys
24
1998 va
2000 destaca
2001
42 milions
7.000 milions
29
11
1845 -
1855
20 anys
1870 va
1874
1877-1893
89 anys
1945 va
2000
8,8 milions
1,19 bilions
135.400 pessetes
132
100
1999
2000
1998
2000
467.800 milions
1999
3,5 milions
1,2 milions
1,7 milions
1,9 milions
2,8 milions
1999
39
12,1 milions
17 d'abril
30.000 milions
46.000 milions
1999
30 dies
2.000 milions
56 milions
1.400 socis
247 llocs
1.400 socis
75 anys
127 pacients
11
90.000 milions
150 anys
28 persones
50 col·laboradors


In [21]:
# Tokenitza les troballes.
# Converteix això en una llista de llistes de tokens en format string.
troballes_tokenitzades = [[t.text for t in nlp(trob)] for trob in quantitats]

# Obtenir les referències com a instàncies d'EntitatNomenada.
entitats_nomenades += convertir_fragments_a_referencies(tokens, troballes_tokenitzades, 'CARDINAL')

# Ara verifiquem si aquestes es corresponen correctament als tokens inicials.
for en in entitats_nomenades:
    en.verificar(tokens)
else:
    print('Tot verificat!')

DESCARTAT: ['742', 'havia']
DESCARTAT: ['638']
DESCARTAT: ['339']
DESCARTAT: ['45']
DESCARTAT: ['000']
DESCARTAT: ['000']
DESCARTAT: ['000']
DESCARTAT: ['000']
DESCARTAT: ['40']
DESCARTAT: ['000']
DESCARTAT: ['132']
DESCARTAT: ['12,1', 'milions']
DESCARTAT: ['8,01', 'milions']
DESCARTAT: ['248.489', 'unitats']
DESCARTAT: ['2004']
DESCARTAT: ['21', 'hores']
DESCARTAT: ['200', 'metres']
DESCARTAT: ['3', "s'", 'ha']
DESCARTAT: ['20', 'hores']
DESCARTAT: ['00', 'hores']
DESCARTAT: ['49']
DESCARTAT: ['202']
DESCARTAT: ['74']
DESCARTAT: ['2000', "l'", '1']
DESCARTAT: ['16']
DESCARTAT: ['159', 'originals']
DESCARTAT: ['13']
DESCARTAT: ['93']
DESCARTAT: ['414']
DESCARTAT: ['37']
DESCARTAT: ['04']
DESCARTAT: ['25']
DESCARTAT: ['18']
DESCARTAT: ['45']
Tot verificat!


In [22]:
# Ordenem les prioritats de cada etiqueta per sobreescriure les EN que apareixen en més d'una RegEx
prioritats = {
    'PERCENT' :     7,
    'MONEY' :       6,
    'DATE' :        5,
    'QUANTITY' :    4,
    'ORDINAL' :     3,
    'TIME' :        2,
    'CARDINAL' :    1,
    'ERROR' :       0,
    'PERCENT-ERROR' :     0,
    'ORDINAL-ERROR' :     0,
    'MONEY-ERROR' :       0,
    'DATE-ERROR' :        0,
    'QUANTITY-ERROR' :    0,
    'TIME-ERROR' :        0,
    'CARDINAL-ERROR' :    0,
    'ERROR-ERROR' :       0
}

# Bucle en què iterem per totes les combinacions d'etiquetes
for i, en1 in enumerate(entitats_nomenades):
    for j, en2 in enumerate(entitats_nomenades):
        
        # Condicional per evitar comparacions entre una mateixa etiqueta
        if i == j:
            continue

        # Condicional per evitar repetir comparacions que ja hem fet
        if i > j:
            continue
        
        # Condicional per assignar la categoria ERROR si etiquetes se solapen
        if comprova_si_solapa(en1, en2):
            if prioritats[en1.etiqueta] <= prioritats[en2.etiqueta]:
                if '-ERROR' not in en1.etiqueta:
                    en1.etiqueta += '-ERROR'
            else: 
                if '-ERROR' not in en2.etiqueta:
                    en2.etiqueta += '-ERROR'

In [23]:
# Iterem per les entitats en sentit contrari per eliminar els errors.
# Ho fem en sentit contrari per evitar errors pel desplaçament dels indexs.
for i in reversed(range(len(entitats_nomenades))):
    
    # Elimina els que han estat marcats com errors.
    if '-ERROR' in entitats_nomenades[i].etiqueta:
        del entitats_nomenades[i]

for en in entitats_nomenades:
    en.print()

DATE (323-326) = ["l'", 'any', '1986']
DATE (743-744) = ['1999']
DATE (932-933) = ['1989']
DATE (940-941) = ['1991']
DATE (951-952) = ['1993']
DATE (959-960) = ['1994']
DATE (967-968) = ['1996']
DATE (975-976) = ['1999']
DATE (2301-2302) = ['1998']
DATE (2670-2675) = ["L'", 'octubre', 'd', 'el', '1998']
DATE (2706-2709) = ['26', 'de', 'desembre']
DATE (2731-2734) = ['24', 'de', 'maig']
DATE (3205-3206) = ['1995']
DATE (3624-3625) = ['13-M']
DATE (3675-3680) = ['31', 'de', 'maig', 'de', '1938']
DATE (4058-4059) = ['1999']
DATE (4499-4502) = ['4', "d'", 'abril']
DATE (4557-4560) = ['29', 'de', 'març']
DATE (4599-4602) = ['4', "d'", 'abril']
DATE (5518-5520) = ['el', '1936']
DATE (5549-5551) = ['el', '1936']
DATE (5776-5777) = ['1859']
DATE (6456-6459) = ['24', 'de', 'juliol']
DATE (7089-7092) = ["l'", 'any', '1998']
DATE (7194-7197) = ["l'", 'any', '2000']
DATE (7255-7260) = ["l'", 'abril', 'd', 'el', '2001']
DATE (7519-7520) = ['1845']
DATE (7521-7522) = ['1902']
DATE (7533-7535) = ['se

In [24]:
# Convertim totes les EN trobades per Spacy al nostre format.
entitats_spacy = [EntitatNomenada(e.label_, e.start, e.end - e.start, tokens[e.start:e.end]) for e in doc_spacy.ents]

In [25]:
DESCARTAR_REGEX = True

# Bucle en què iterem per totes les combinacions d'etiquetes nostres i de spacy
for i, en_spacy in enumerate(entitats_spacy):
    for j, en_regex in enumerate(entitats_nomenades):
        
        # Condicional per assignar la categoria ERROR si etiquetes se solapen
        if comprova_si_solapa(en_spacy, en_regex):
            
            if DESCARTAR_REGEX:

                en_regex.etiqueta += '-ERROR'

            else:

                en_spacy.etiqueta += '-ERROR'

# L'afegim a la llista general d'EN.
for i, en_spacy in enumerate(entitats_spacy):
    entitats_nomenades.append(en_spacy)
                

In [26]:
# Iterem per les entitats en sentit contrari per eliminar els errors.
# Ho fem en sentit contrari per evitar errors pel desplaçament dels indexs.
for i in reversed(range(len(entitats_nomenades))):
    
    # Elimina els que han estat marcats com errors.
    if '-ERROR' in entitats_nomenades[i].etiqueta:
        del entitats_nomenades[i]

In [27]:
# Eliminem totes les EN perquè les hem inclòs a la nostra llista.
doc_spacy.ents = []

# Afegim totes les EN que hem trobat i/o no hem descartat.
for entitat in entitats_nomenades:
    afegir_entitat_nomenada(doc_spacy, entitat)

In [28]:
# Guardem la variable en un arxiu Pickle.
with open('entitats_nomenades.corpus_raw.pickle', 'wb') as file:
    pickle.dump(entitats_nomenades, file)

# Guardem la variable en un arxiu Pickle.
with open('tokens.corpus_raw.pickle', 'wb') as file:
    pickle.dump(tokens, file)

In [29]:
html = displacy.render(doc_spacy, style="ent", jupyter=not SAVE_IMAGES)

if SAVE_IMAGES:
    output_path = Path("C:\\Users\\alber\\OneDrive\\Documents\\Cosetes\\prova_imatge1.html")
    output_path.open("w", encoding="utf-8").write(html)